In [1]:
import os
import numpy as np
import pretty_midi
from midi_features import get_track_features
from train import parse_melody_channel_txt

In [2]:
def get_dataset(label_txt_path, midi_dir):
    all_track_features = None
    all_labels = None
    filenames, channels = parse_melody_channel_txt(label_txt_path)
    for i_file in range(len(filenames)):
        midi_path = os.path.join(midi_dir, filenames[i_file])
        if not os.path.exists(midi_path):
            continue
        pretty_midi_features = pretty_midi.PrettyMIDI(midi_path)
        num_channels = len(pretty_midi_features.instruments)
        labels = np.zeros(num_channels, bool)
        labels[channels[i_file]] = True
        track_features = get_track_features(pretty_midi_features)
        all_labels = labels if all_labels is None else np.concatenate((all_labels, labels), axis=0)
        all_track_features = track_features if all_track_features is None else np.concatenate((all_track_features, track_features), axis=0)
    return all_track_features, all_labels

In [3]:
label_txt_path = "melody_channel_ids_new.txt"
midi_dir = os.path.join("dataset_cmu_melody", "train")
X, y = get_dataset(label_txt_path, midi_dir)
print(X.shape, y.shape)

D:\softwares\conda\miniconda3\envs\midiaug\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


(581, 18) (581,)


In [4]:
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

normalizer = StandardScaler().fit(X)
X_normed = normalizer.transform(X)

# feat_mean, feat_std = X.mean(axis = 0), X.std(axis = 0)
# X_normed = (X - feat_mean) / feat_std

clf0 = svm.SVC().fit(X_normed, y)
clf1 = tree.DecisionTreeClassifier().fit(X_normed, y)
clf2 = RandomForestClassifier(n_estimators=100, random_state=1).fit(X_normed, y)
clf3 = HistGradientBoostingClassifier(max_iter=100).fit(X_normed, y)

In [5]:
eclf = VotingClassifier(
    estimators=[('svm', clf1), ('tree', clf1), ('rf', clf2), ('gb', clf3)],
    voting='hard'
)

In [6]:
for clf, label in zip([clf0, clf1, clf2, clf3, eclf], ['SVC', 'DecisionTree', 'RandomForest', 'HistGradientBoosting', 'Ensemble']):
    scores = cross_val_score(clf, X_normed, y, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.93 (+/- 0.01) [SVC]
Accuracy: 0.95 (+/- 0.03) [DecisionTree]
Accuracy: 0.95 (+/- 0.02) [RandomForest]
Accuracy: 0.97 (+/- 0.02) [HistGradientBoosting]
Accuracy: 0.96 (+/- 0.03) [Ensemble]


In [7]:
pipeline = Pipeline([
    ('normalizer', StandardScaler()), 
    ('gb_classifier', HistGradientBoostingClassifier(max_iter=100))
]).fit(X, y)
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=5)
print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'HistGradientBoosting'))

y_pred = pipeline.predict(X)
print((y_pred == y).sum() / len(y))

Accuracy: 0.97 (+/- 0.02) [HistGradientBoosting]
1.0


In [ ]:
y